In [1]:
# Import all the things

import json
import pandas as pd
import numpy as np
import datetime
import acquire
from prepare import basic_clean, tokenize, lemmatize, stem, remove_stopwords
import prepare
import wrangle 
import re
from sklearn.feature_extraction.text import TfidfVectorizer

from wordcloud import WordCloud
from matplotlib import pyplot as plt
import seaborn as sns

import nltk

from scipy import stats

import warnings
warnings.filterwarnings("ignore")

### Acquire

merge host_map to utterances

In [21]:
# df = pd.read_csv('./utterances.csv')
# ep_df = pd.read_csv('./episodes.csv')


In [3]:
# df['speaker'] = df.speaker.str.lower()
# df['program'] = df.program.str.lower()
# df['title'] = df.title.str.lower()

In [23]:
# # joining utterances df ('df') and episodes on 'id'
# joined_df = pd.merge(df, ep_df, left_on = 'episode', right_on='id', how = 'inner')
# joined_df.drop(columns = ['id'], inplace=True)
# joined_df.rename(columns={'episode':'episode_id'}, inplace = True)

In [24]:
# joined_df['is_host'] = joined_df.speaker.str.contains(r'\W*(host)\W*')

In [7]:
df = acquire.get_df()

,episode_id,episode_order,speaker,utterance,program,title,episode_date,is_host
0,57264,9,"ms. loren mooney (editor-in-chief, bicycling m...","It's a 2,200-mile race. To give some sense of ...",talk of the nation,how to watch the tour de france,2010-07-12,False
1,57264,10,"ms. loren mooney (editor-in-chief, bicycling m...",So for a top competitor like Lance to try to m...,talk of the nation,how to watch the tour de france,2010-07-12,False
2,57264,11,"neal conan, host","So in every team, presumably there's one star,...",talk of the nation,how to watch the tour de france,2010-07-12,True
3,57264,12,"ms. loren mooney (editor-in-chief, bicycling m...",That's right. Each team has nine riders. And w...,talk of the nation,how to watch the tour de france,2010-07-12,False
4,57264,13,"neal conan, host","So slipstream, this is like drafting in car ra...",talk of the nation,how to watch the tour de france,2010-07-12,True


In [8]:
df_test.shape

(3199858, 8)

## Preparation

- nulls
- duplicates
- create sentiment score
- use prep functions
- keep some punctuation
- split
- date to datetime

In [4]:

# obtain top 10 hosts
hosts_to_keep = df[df.is_host == True].speaker.value_counts().head(10).index.to_list()
# create host df
hosts_df = df[df.speaker.isin(hosts_to_keep)]
# get episode_id of top 10 hosts
top_host_episodes = hosts_df.episode_id.value_counts().index.to_list()
# create dataframe with mask of episodes with top hosts
df = df[df.episode_id.isin(top_host_episodes)]
# remove rows with foreign languages spoken
df = df[df.utterance!='(Foreign language spoken)']
# remove rows without speaker (sound effects)
df = df[df.speaker!='_NO_SPEAKER']
# drop duplicates
df.drop_duplicates(inplace = True)
# drop nulls
df.dropna(inplace=True)
# create clean column
df['clean'] = [tokenize(basic_clean(u)) for u in df.utterance]
# create lemmatized column
df['lemmatized'] = df['clean'].apply(tokenize).apply(lemmatize)
# vader sentiment analysis
sia = nltk.sentiment.SentimentIntensityAnalyzer()
df['vader'] = df.lemmatized.apply(lambda doc: sia.polarity_scores(doc)['compound'])
# date column to datetime
df['date'] = pd.to_datetime(df.episode_date)
# cutoff dates prior to 2005 due to low observation count
df = df[df.date > '2005']
# double check drop nulls
df.dropna(inplace = True)

In [9]:
df.speaker.value_counts()

neal conan, host               255143
_no_speaker                    109693
ira flatow, host                73188
steve inskeep, host             59158
robert siegel, host             49496
                                ...  
mr. fu                              1
lila thompson                       1
mr. vic vickers (candidate)         1
sandy luger                         1
alvaro villarueda                   1
Name: speaker, Length: 114886, dtype: int64

In [ ]:
df = prep_npr_data(df)

In [27]:
# obtain top 10 hosts
hosts_to_keep = df[df.is_host == True].speaker.value_counts().head(10).index.to_list()
# create host df
hosts_df = df[df.speaker.isin(hosts_to_keep)]
# get episode_id of top 10 hosts
top_host_episodes = hosts_df.episode_id.value_counts().index.to_list()


In [30]:
# create dataframe with mask of episodes with top hosts
df = df[df.episode_id.isin(top_host_episodes)]

In [31]:
# drop nulls
df.dropna(inplace=True)

In [32]:
# drop duplicates
df.drop_duplicates(inplace = True)

In [33]:
# remove rows without speaker (sound effects)
df = df[df.speaker!='_NO_SPEAKER']

In [34]:
# remove rows with foreign languages spoken
df = df[df.utterance!='(Foreign language spoken)']


In [35]:
# create clean column
df['clean'] = [tokenize(basic_clean(u)) for u in df.utterance]


In [10]:
# create lemmatized column
df['lemmatized'] = df['clean'].apply(tokenize).apply(lemmatize)


In [ ]:
df.head()

In [2]:
df = wrangle.get_npr_data()

Getting top hosts...
Getting Episode ID's for the hosts...
Double checking speaker variables...
Dropping duplicates...
Dropping null values...
Cleaning corpus...
Lemmatizing corpus...
Analyzing sentiment with VADER...
Converting to datetime...
Trimming timeline...
The df has 1889857 rows and 12 columns.


In [3]:
df.shape

(1889857, 12)

In [4]:
df.head()

,episode_id,episode_order,speaker,utterance,program,title,episode_date,is_host,clean,lemmatized,vader,date
0,57264,9,"ms. loren mooney (editor-in-chief, bicycling m...","It's a 2,200-mile race. To give some sense of ...",talk of the nation,how to watch the tour de france,2010-07-12,False,"it s a 2,200 mile race. to give some sense of ...","it s a 2,200 mile race. to give some sense of ...",0.0000,2010-07-12
1,57264,10,"ms. loren mooney (editor-in-chief, bicycling m...",So for a top competitor like Lance to try to m...,talk of the nation,how to watch the tour de france,2010-07-12,False,so for a top competitor like lance to try to m...,so for a top competitor like lance to try to m...,0.9346,2010-07-12
2,57264,11,"neal conan, host","So in every team, presumably there's one star,...",talk of the nation,how to watch the tour de france,2010-07-12,True,"so in every team , presumably there s one star...","so in every team , presumably there s one star...",0.7096,2010-07-12
3,57264,12,"ms. loren mooney (editor-in-chief, bicycling m...",That's right. Each team has nine riders. And w...,talk of the nation,how to watch the tour de france,2010-07-12,False,that s right. each team has nine riders. and w...,that s right. each team ha nine riders. and wh...,0.9274,2010-07-12
4,57264,13,"neal conan, host","So slipstream, this is like drafting in car ra...",talk of the nation,how to watch the tour de france,2010-07-12,True,"so slipstream , this is like drafting in car r...","so slipstream , this is like drafting in car r...",0.3612,2010-07-12


In [8]:
df[df.is_host == True].speaker.value_counts().index.to_list()

['neal conan, host',
 'ira flatow, host',
 'steve inskeep, host',
 'robert siegel, host',
 'melissa block, host',
 'renee montagne, host',
 'farai chideya, host',
 'scott simon, host',
 'rachel martin, host',
 'david greene, host',
 'michele norris, host',
 'audie cornish, host',
 'noel king, host',
 'linda wertheimer, host',
 'ari shapiro, host',
 'mary louise kelly, host',
 'guy raz, host',
 'deborah amos, host',
 'madeleine brand, host',
 'liane hansen, host',
 'lynn neary, host',
 'kelly mcevers, host',
 'john ydstie, host',
 'ed gordon, host',
 'ailsa chang, host',
 'tony cox, host',
 'don gonyea, host',
 'mr. mike rowe (host, "dirty jobs)',
 'flora lichtman, host',
 'rebecca roberts, host',
 'ira glass (host)',
 'mr. jeff obafemi carr (radio host, freestyle)',
 'mr. jeff obafemi carr (host, freestyle)',
 'ms. nigella lawson (host/author, "nigella express")',
 'susan stamberg, host',
 'mr. jeff obafemi carr (host, "freestyle")',
 'mr. jeff obafemi carr (host, “freestyle”)',
 'mr. 

## Initial Planning Analysis

In [19]:
# read in primary utterances csv for dataframe
df = pd.read_csv('./utterances.csv')
df.head()

,episode,episode_order,speaker,utterance
0,57264,9,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...","It's a 2,200-mile race. To give some sense of ..."
1,57264,10,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...",So for a top competitor like Lance to try to m...
2,57264,11,"NEAL CONAN, host","So in every team, presumably there's one star,..."
3,57264,12,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...",That's right. Each team has nine riders. And w...
4,57264,13,"NEAL CONAN, host","So slipstream, this is like drafting in car ra..."


In [20]:
# find utterance count per episode
df.episode.value_counts()

19633    569
35108    552
57481    539
73336    492
66523    480
        ... 
80400      1
61102      1
88047      1
73497      1
92554      1
Name: episode, Length: 104920, dtype: int64

In [21]:
# find utterance count per episode
test = df.episode.value_counts() < 2
test.sum()

1357

In [22]:
# pull in episodes df
ep_df = pd.read_csv('./episodes.csv')
ep_df.head()

,id,program,title,episode_date
0,98814,Morning Edition,Senate Ushers In New Year With 'Fiscal Cliff' ...,2013-01-01
1,98824,Morning Edition,Cheap Bubbly Or Expensive Sparkling Wine? Look...,2012-12-31
2,98821,Morning Edition,U.S. Gas Prices Reach Record Level In 2012,2013-01-01
3,98806,Morning Edition,House Approves 'Fiscal Cliff' Measure,2013-01-02
4,98823,Morning Edition,The Fiscal Cliff: A Love Story,2012-12-31


In [23]:
# check episode content to see if it matches episode id
df[df.episode == 92554].utterance.to_list()

["Good morning. I'm David Greene. Here's the weather forecast for Albuquerque, New Mexico today - cloudy with a chance of grasshoppers. Yes, that's right. The animals are swarming the city so densely, local radar is mistaking them for rain crowds. These insects are hovering in masses as high as a thousand feet up. Officials say this is the worst infestation in 20 years, but it should pass soon. With any luck, it will go back to raining rain - helping to clean up the mess left behind by these grasshoppers. You're listening to MORNING EDITION."]

In [24]:
# check ep_df.id = df.episode
ep_df[ep_df.id == 92554].title.to_list()

["Albuquerque's Weather? Cloudy With A Chance of Grasshoppers"]

### Initial takeaways
There are episodes with one utterance, showing less interview setting and more reporting.

ep_df.id == df.episode

## Sentiment Analysis

In [3]:
import nltk.sentiment
sia = nltk.sentiment.SentimentIntensityAnalyzer()

In [20]:
df['vader'] = df.lemmatized.apply(lambda doc: sia.polarity_scores(doc)['compound'])

In [34]:
df['date'] = pd.to_datetime(df.episode_date)

In [36]:
df.drop(columns = 'episode_date', inplace = True)

In [37]:
df.head()

,episode_id,episode_order,speaker,utterance,program,title,is_host,clean,lemmatized_stopped,vader_stopped,date
0,57264,9,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...","It's a 2,200-mile race. To give some sense of ...",Talk of the Nation,How To Watch The Tour De France,False,it s a 2200 mile race. to give some sense of p...,2200 mile race. give sense perspective roughly...,0.0000,2010-07-12
1,57264,10,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...",So for a top competitor like Lance to try to m...,Talk of the Nation,How To Watch The Tour De France,False,so for a top competitor like lance to try to m...,top competitor like lance try make much time 1...,0.9313,2010-07-12
2,57264,11,"NEAL CONAN, host","So in every team, presumably there's one star,...",Talk of the Nation,How To Watch The Tour De France,True,so in every team presumably there s one star o...,every team presumably one star one equivalent ...,0.7096,2010-07-12
3,57264,12,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...",That's right. Each team has nine riders. And w...,Talk of the Nation,How To Watch The Tour De France,False,that s right. each team has nine riders. and w...,right. team ha nine riders. basically team lea...,0.9274,2010-07-12
4,57264,13,"NEAL CONAN, host","So slipstream, this is like drafting in car ra...",Talk of the Nation,How To Watch The Tour De France,True,so slipstream this is like drafting in car rac...,slipstream like drafting car racing right ?,0.3612,2010-07-12


In [40]:
df.to_csv('npr_corpus.csv', index = False)

In [4]:
df = wrangle.get_npr_data()

The df has 2998347 rows and 10 columns.


In [22]:
test = df[df.date < '2004']

In [46]:
df[df.date < '2005'].shape

(380, 10)

## JOSH
- Are there words that are said more frequently by hosts? By time of day? By category?
What host(s) say(s) the most words?

In [32]:
df = wrangle.get_npr_data()

The df has 2998347 rows and 10 columns.


In [3]:
train, validate, test = wrangle.split_data(df)

### Hosts

In [53]:
df[df.is_host == True].groupby('speaker')['lemmatized_stopped'].apply(lambda x: ','.join(x)).reset_index()


,speaker,lemmatized_stopped
0,"a l, host",lot praise interview mary louise last week pun...
1,"a martinez, host",harvey downgraded hurricane tropical storm con...
2,"aarti shahani, host",u.s. china trade deal may approaching final le...
3,"adam davidson, host",well important realize long long time. since g...
4,agunda (hostage),interpreter second day boyviki said thing like...
...,...,...
865,"wade goodwyn, host",brought change bail system new york city speci...
866,"wetheimer, host",charlie chan novel came time united state wa i...
867,when the president hosted lawmakers from both ...,"televised summit .,mr. obama invited republica..."
868,"william zinsser, host",suddenly family wa dysfunctional memory squali...


In [16]:
speaker_words = {'speaker':[], 'words':[]}
for s in train[train.is_host == True].speaker:
    w = ' '.join(train[train.speaker == s].lemmatized_stopped)
    speaker_words['speaker'].append(s)
    speaker_words['words'].append(w)

KeyboardInterrupt: 

In [15]:
speaker_words

{'speaker': ['episode_order',
  'speaker',
  'utterance',
  'program',
  'title',
  'is_host',
  'clean',
  'lemmatized_stopped',
  'vader_stopped',
  'date'],
 'words': ['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'court ruling far applies specifically called electioneering communications. addressed nonprofit group make called independent expenditure campaign covered different section campaign finance law . 00 900 pm unidentified broadcaster 1 gop brand sustaining serious damage shutdown ... start build nest egg start invest equipment thing much invest people biggest resource much going difficult . pelosi also appointed adam smith washington state known sober pro military moderate. ranking democrat house armed service committee. named tammy duckworth illinois army national guard veteran lost leg iraq war . unidentified broadcaster 3 criticism shutdown focused republican ... sort language boehner using shutdown had. s.v. date npr news . sheila krumholz run center responsive p

In [33]:
host_df = df[df.is_host==True]
hosts_with_the_most = host_df.speaker.value_counts().head(10).index.to_list()


In [19]:
top_hosts_df

,episode_order,speaker,utterance,program,title,is_host,clean,lemmatized_stopped,vader_stopped,date
episode_id,,,,,,,,,,
31179,17,"farai chideya, host",Dr. Warren Johnson heads the division of inter...,News & Notes,Doubts About Haiti and the Spread of HIV/AIDS,True,dr. warren johnson heads the division of inter...,dr. warren johnson head division international...,0.0000,2007-11-12
9770,9,"melissa block, host",Success is right. Another video posted at the ...,Weekend Edition Saturday,Need A Useless Robot? Simone Giertz Is The Queen,True,success is right. another video posted at the ...,success right. another video posted time break...,0.7269,2016-03-19
57481,184,"neal conan, host","Unidentified Man #2: Well, what about ""The God...",Talk of the Nation,"'Completely Hollywood': 197 Films, Abridged",True,unidentified man 2 well what about the godfath...,unidentified man 2 well godfather ? godfather ?,0.3527,2010-06-24
118798,2,"melissa block, host","The three-term incumbent, Democrat Patty Murra...",All Things Considered,A Toss-Up In Washington State Senate Race,True,the three term incumbent democrat patty murray...,three term incumbent democrat patty murray cha...,-0.4767,2010-09-28
64604,68,"rachel martin, host","OK, When you have the answer, go to our websit...",Weekend Edition Sunday,"Brave Sir Robin Ran Away, But The Puzzle Is St...",True,ok when you have the answer go to our website ...,ok answer go website npr.orgpuzzle click submi...,0.9081,2012-05-06
...,...,...,...,...,...,...,...,...,...,...
93662,3,"rachel martin, host",You're someone who has studied this relationsh...,All Things Considered,Russian Relations 'Extremely Tense' As U.S. Se...,True,you re someone who has studied this relationsh...,someone ha studied relationship long time. ten...,0.0258,2015-06-23
134977,4,"robert siegel, host","Tomorrow, President Bush is expected to visit ...",All Things Considered,Tornado Victims in Kansas Still Reel from Storm,True,tomorrow president bush is expected to visit g...,tomorrow president bush expected visit greensb...,-0.8720,2007-05-08
116711,62,"robert siegel, host",That's with cup holders included.,All Things Considered,What Recession? It's Boom Time For Nebraska Farms,True,that s with cup holders included .,cup holder included .,0.0000,2011-02-25


In [34]:
hosts_with_the_most

['neal conan, host',
 'ira flatow, host',
 'steve inskeep, host',
 'robert siegel, host',
 'melissa block, host',
 'renee montagne, host',
 'farai chideya, host',
 'scott simon, host',
 'rachel martin, host',
 'david greene, host']